Reference:  
·&nbsp;https://www.kaggle.com/code/gusthema/asl-fingerspelling-recognition-w-tensorflow#Data-visualization-using-mediapipe-APIs   
·&nbsp;https://www.kaggle.com/code/markwijkhuizen/aslfr-eda-preprocessing-dataset  
·&nbsp;https://www.kaggle.com/code/markwijkhuizen/aslfr-transformer-training-inference#Verify-No-NaN-Predictions  

In [62]:
%%capture
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
base_input_path='/kaggle/input/asl-fingerspelling/'
base_output_path='/kaggle/working/'

In [63]:
#!pip install mediapipe

In [64]:
import warnings
warnings.filterwarnings("ignore" , category=UserWarning)
import shutil
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import tensorflow as tf
import json
#import mediapipe
import matplotlib
import matplotlib.pyplot as plt
import random
import re

from skimage.transform import resize
#from mediapipe.framework.formats import landmark_pb2
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
from matplotlib import animation, rc

tqdm.pandas()

print("TensorFlow v" + tf.__version__)
#print("Mediapipe v" + mediapipe.__version__)

TensorFlow v2.12.0


In [65]:
with open(base_input_path+'character_to_prediction_index.json') as json_file:
    CHAR2ORD = json.load(json_file)
    
ORD2CHAR = {j:i for i,j in CHAR2ORD.items()}
print(f'{len(ORD2CHAR)}\n{ORD2CHAR}')

# Character to Ordinal Encoding Mapping   
pd.Series(CHAR2ORD).to_frame('Ordinal Encoding').head()

59
{0: ' ', 1: '!', 2: '#', 3: '$', 4: '%', 5: '&', 6: "'", 7: '(', 8: ')', 9: '*', 10: '+', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '=', 28: '?', 29: '@', 30: '[', 31: '_', 32: 'a', 33: 'b', 34: 'c', 35: 'd', 36: 'e', 37: 'f', 38: 'g', 39: 'h', 40: 'i', 41: 'j', 42: 'k', 43: 'l', 44: 'm', 45: 'n', 46: 'o', 47: 'p', 48: 'q', 49: 'r', 50: 's', 51: 't', 52: 'u', 53: 'v', 54: 'w', 55: 'x', 56: 'y', 57: 'z', 58: '~'}


,Ordinal Encoding
,0
!,1
#,2
$,3
%,4


In [66]:
# Set to values that are not within the CHAR2ORD integer range
PAD_TOKEN = len(CHAR2ORD) # Padding
SOS_TOKEN = len(CHAR2ORD) + 1 # Start Of Sentence
EOS_TOKEN = len(CHAR2ORD) + 2 # End Of Sentence

N_UNIQUE_CHARACTERS0 = len(CHAR2ORD)
N_UNIQUE_CHARACTERS = len(CHAR2ORD) + 1 + 1 + 1

In [67]:
dataset_df_path=f'{base_input_path}train.csv'
dataset_df = pd.read_csv(dataset_df_path)
print("Full train dataset shape is {}".format(dataset_df.shape))
dataset_df.head()

Full train dataset shape is (67208, 5)


,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [68]:
"""
The datasets contain addresses/phone numbers/urls (This is written in the official description file)
"""
def get_phrase_type(phrase):
    # Phone Number
    if re.match(r'^[\d+-]+$', phrase):
        return 'phone_number'
    # url
    elif any([substr in phrase for substr in ['www', '.', '/']]) and ' ' not in phrase:
        return 'url'
    # Address
    else:
        return 'address'
    
dataset_df['phrase_type'] = dataset_df['phrase'].apply(get_phrase_type)
dataset_df.head()


,path,file_id,sequence_id,participant_id,phrase,phrase_type
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,address
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,url
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,address
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,address
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,address


In [69]:
# Split Phrase To Char Tuple
dataset_df['phrase_char'] = dataset_df['phrase'].apply(tuple)
# Character Length of Phrase
dataset_df['phrase_char_len'] = dataset_df['phrase_char'].apply(len)
dataset_df.head()

,path,file_id,sequence_id,participant_id,phrase,phrase_type,phrase_char,phrase_char_len
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,address,"(3, , c, r, e, e, k, h, o, u, s, e)",12
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,url,"(s, c, a, l, e, s, /, k, u, h, a, y, l, a, h)",15
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,address,"(1, 3, 8, 3, , w, i, l, l, i, a, m, , l, a, ...",19
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,address,"(9, 8, 8, , f, r, a, n, k, l, i, n, , l, a, ...",17
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,address,"(6, 9, 2, 0, , n, o, r, t, h, e, a, s, t, , ...",25


In [70]:
def encode_phrase(phrase):
    # .get(char, -1): if the char doesn't exist in dict, return -1.
    return [CHAR2ORD.get(char, -1) for char in phrase]

dataset_df['ordinal_encoding'] = dataset_df['phrase'].apply(encode_phrase)

dataset_df.head()

,path,file_id,sequence_id,participant_id,phrase,phrase_type,phrase_char,phrase_char_len,ordinal_encoding
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,address,"(3, , c, r, e, e, k, h, o, u, s, e)",12,"[18, 0, 34, 49, 36, 36, 42, 39, 46, 52, 50, 36]"
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,url,"(s, c, a, l, e, s, /, k, u, h, a, y, l, a, h)",15,"[50, 34, 32, 43, 36, 50, 14, 42, 52, 39, 32, 5..."
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,address,"(1, 3, 8, 3, , w, i, l, l, i, a, m, , l, a, ...",19,"[16, 18, 23, 18, 0, 54, 40, 43, 43, 40, 32, 44..."
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,address,"(9, 8, 8, , f, r, a, n, k, l, i, n, , l, a, ...",17,"[24, 23, 23, 0, 37, 49, 32, 45, 42, 43, 40, 45..."
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,address,"(6, 9, 2, 0, , n, o, r, t, h, e, a, s, t, , ...",25,"[21, 24, 17, 15, 0, 45, 46, 49, 51, 39, 36, 32..."


In [71]:
# Add complete file path
def get_file_path(path):
    return f'{base_input_path}{path}'

dataset_df['file_path'] = dataset_df['path'].apply(get_file_path)
dataset_df.head()

,path,file_id,sequence_id,participant_id,phrase,phrase_type,phrase_char,phrase_char_len,ordinal_encoding,file_path
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,address,"(3, , c, r, e, e, k, h, o, u, s, e)",12,"[18, 0, 34, 49, 36, 36, 42, 39, 46, 52, 50, 36]",/kaggle/input/asl-fingerspelling/train_landmar...
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,url,"(s, c, a, l, e, s, /, k, u, h, a, y, l, a, h)",15,"[50, 34, 32, 43, 36, 50, 14, 42, 52, 39, 32, 5...",/kaggle/input/asl-fingerspelling/train_landmar...
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,address,"(1, 3, 8, 3, , w, i, l, l, i, a, m, , l, a, ...",19,"[16, 18, 23, 18, 0, 54, 40, 43, 43, 40, 32, 44...",/kaggle/input/asl-fingerspelling/train_landmar...
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,address,"(9, 8, 8, , f, r, a, n, k, l, i, n, , l, a, ...",17,"[24, 23, 23, 0, 37, 49, 32, 45, 42, 43, 40, 45...",/kaggle/input/asl-fingerspelling/train_landmar...
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,address,"(6, 9, 2, 0, , n, o, r, t, h, e, a, s, t, , ...",25,"[21, 24, 17, 15, 0, 45, 46, 49, 51, 39, 36, 32...",/kaggle/input/asl-fingerspelling/train_landmar...


In [72]:
from sklearn.preprocessing import OneHotEncoder

def do_OneHotEncoder(data, col):
    encoder = OneHotEncoder(sparse_output=False)

    phrase_type = data[col].values.reshape(-1, 1)

    phrase_type_onehot = encoder.fit_transform(phrase_type)
    phrase_type_onehot_df = pd.DataFrame(phrase_type_onehot, columns=encoder.get_feature_names_out(['phrase_type']))

    data = data.drop(columns=[col])
    data = pd.concat([data.reset_index(drop=True), phrase_type_onehot_df], axis=1)

    return data
    
dataset_df=do_OneHotEncoder(dataset_df,'phrase_type')
dataset_df.head()

,path,file_id,sequence_id,participant_id,phrase,phrase_char,phrase_char_len,ordinal_encoding,file_path,phrase_type_address,phrase_type_phone_number,phrase_type_url
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,"(3, , c, r, e, e, k, h, o, u, s, e)",12,"[18, 0, 34, 49, 36, 36, 42, 39, 46, 52, 50, 36]",/kaggle/input/asl-fingerspelling/train_landmar...,1.0,0.0,0.0
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,"(s, c, a, l, e, s, /, k, u, h, a, y, l, a, h)",15,"[50, 34, 32, 43, 36, 50, 14, 42, 52, 39, 32, 5...",/kaggle/input/asl-fingerspelling/train_landmar...,0.0,0.0,1.0
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,"(1, 3, 8, 3, , w, i, l, l, i, a, m, , l, a, ...",19,"[16, 18, 23, 18, 0, 54, 40, 43, 43, 40, 32, 44...",/kaggle/input/asl-fingerspelling/train_landmar...,1.0,0.0,0.0
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,"(9, 8, 8, , f, r, a, n, k, l, i, n, , l, a, ...",17,"[24, 23, 23, 0, 37, 49, 32, 45, 42, 43, 40, 45...",/kaggle/input/asl-fingerspelling/train_landmar...,1.0,0.0,0.0
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,"(6, 9, 2, 0, , n, o, r, t, h, e, a, s, t, , ...",25,"[21, 24, 17, 15, 0, 45, 46, 49, 51, 39, 36, 32...",/kaggle/input/asl-fingerspelling/train_landmar...,1.0,0.0,0.0


In [73]:
print(', '.join(dataset_df.columns))

path, file_id, sequence_id, participant_id, phrase, phrase_char, phrase_char_len, ordinal_encoding, file_path, phrase_type_address, phrase_type_phone_number, phrase_type_url


# Create X, y datasets

In [74]:
# The numbers come from: https://developers.google.com/mediapipe/solutions/vision/pose_landmarker
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

In [75]:
X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE]
FEATURE_COLUMNS = X + Y + Z
FEATURE_COLUMNS

['x_right_hand_0',
 'x_right_hand_1',
 'x_right_hand_2',
 'x_right_hand_3',
 'x_right_hand_4',
 'x_right_hand_5',
 'x_right_hand_6',
 'x_right_hand_7',
 'x_right_hand_8',
 'x_right_hand_9',
 'x_right_hand_10',
 'x_right_hand_11',
 'x_right_hand_12',
 'x_right_hand_13',
 'x_right_hand_14',
 'x_right_hand_15',
 'x_right_hand_16',
 'x_right_hand_17',
 'x_right_hand_18',
 'x_right_hand_19',
 'x_right_hand_20',
 'x_left_hand_0',
 'x_left_hand_1',
 'x_left_hand_2',
 'x_left_hand_3',
 'x_left_hand_4',
 'x_left_hand_5',
 'x_left_hand_6',
 'x_left_hand_7',
 'x_left_hand_8',
 'x_left_hand_9',
 'x_left_hand_10',
 'x_left_hand_11',
 'x_left_hand_12',
 'x_left_hand_13',
 'x_left_hand_14',
 'x_left_hand_15',
 'x_left_hand_16',
 'x_left_hand_17',
 'x_left_hand_18',
 'x_left_hand_19',
 'x_left_hand_20',
 'x_pose_13',
 'x_pose_15',
 'x_pose_17',
 'x_pose_19',
 'x_pose_21',
 'x_pose_14',
 'x_pose_16',
 'x_pose_18',
 'x_pose_20',
 'x_pose_22',
 'y_right_hand_0',
 'y_right_hand_1',
 'y_right_hand_2',
 'y_

In [96]:
def reset_hand_pose_index():
    global X_IDX, Y_IDX, Z_IDX, RHAND_IDX, LHAND_IDX, RPOSE_IDX, LPOSE_IDX, OTHER_FEATURES_IDX, FEATURE_COLUMNS
    
    X_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "x_" in col]
    Y_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "y_" in col]
    Z_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "z_" in col]

    RHAND_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "right" in col]
    LHAND_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "left" in col]
    RPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col[-2:]) in RPOSE]
    LPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col[-2:]) in LPOSE]

    hand_pose_indices = set(X_IDX + Y_IDX + Z_IDX + RHAND_IDX + LHAND_IDX + RPOSE_IDX + LPOSE_IDX)
    
    all_indices = set(range(len(FEATURE_COLUMNS)))
    
    OTHER_FEATURES_IDX = list(all_indices - hand_pose_indices)
    

reset_hand_pose_index()

In [77]:
#* problem: not sure why set to 128 here
# Set length of frames to 128
FRAME_LEN = 128

# Create directory to store the new data
if not os.path.isdir("preprocessed"):
    os.mkdir("preprocessed")
else:
    shutil.rmtree("preprocessed")
    os.mkdir("preprocessed")

# Loop through each file_id
for file_id in tqdm(dataset_df.file_id.unique()):
    
    pq_file_path=f"{base_input_path}train_landmarks/{file_id}.parquet"
    file_df = dataset_df.loc[dataset_df["file_id"] == file_id]
    parquet_df = pq.read_table(pq_file_path,columns=['sequence_id'] + FEATURE_COLUMNS).to_pandas()
    tf_file = f"preprocessed/{file_id}.tfrecord"
    parquet_numpy = parquet_df.to_numpy()
    
    # Initialize the pointer to write the output of 
    # each `for loop` below as a sequence into the file.
    with tf.io.TFRecordWriter(tf_file) as file_writer:
        # Loop through each sequence in file.
        for index, row in file_df.iterrows():
#             print(index, row)
#             break
            seq_id, phrase = row['sequence_id'], row['phrase']
            frames = parquet_numpy[parquet_df.index == seq_id]
            
            # Additional features
            additional_features = {
                "phrase_char_len": row['phrase_char_len'],
                "phrase_type_address": row['phrase_type_address'],
                "phrase_type_phone_number": row['phrase_type_phone_number'],
                "phrase_type_url": row['phrase_type_url']
            }
            

            features = {FEATURE_COLUMNS[i]: tf.train.Feature(
                float_list=tf.train.FloatList(value=frames[:, i])) for i in range(len(FEATURE_COLUMNS))}
            features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(phrase, 'utf-8')]))

            # Add additional features as float
            for key, value in additional_features.items():
                features[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

            record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
            file_writer.write(record_bytes)
#         # Loop through each sequence in file.
#         for seq_id, phrase in zip(file_df.sequence_id, file_df.phrase):
#             frames = parquet_numpy[parquet_df.index == seq_id]
            
#             # Calculate the number of NaN values in each hand landmark
#             r_nonan = np.sum(np.sum(np.isnan(frames[:, RHAND_IDX]), axis = 1) == 0)
#             l_nonan = np.sum(np.sum(np.isnan(frames[:, LHAND_IDX]), axis = 1) == 0)
#             no_nan = max(r_nonan, l_nonan)
            
#             if 2*len(phrase)<no_nan:
#                 features = {FEATURE_COLUMNS[i]: tf.train.Feature(
#                     float_list=tf.train.FloatList(value=frames[:, i])) for i in range(len(FEATURE_COLUMNS))}
#                 features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(phrase, 'utf-8')]))
#                 record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
#                 file_writer.write(record_bytes)

  0%|          | 0/68 [00:00<?, ?it/s]

In [78]:
raw_dataset = tf.data.TFRecordDataset(f'{base_output_path}preprocessed/532011803.tfrecord')

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "phrase"
    value {
      bytes_list {
        value: "2801 etters"
      }
    }
  }
  feature {
    key: "phrase_char_len"
    value {
      float_list {
        value: 11.0
      }
    }
  }
  feature {
    key: "phrase_type_address"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "phrase_type_phone_number"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "phrase_type_url"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "x_left_hand_0"
    value {
      float_list {
        value: 0.7366787791252136
        value: 0.7279726266860962
        value: 0.7640392780303955
        value: nan
        value: nan
        value: nan
        value: nan
        value: nan
        value: nan
        value: nan
        value: nan
        value: nan
        value: nan
        value: 0.728546142578125
        value: 0.687822163105011
        value

In [97]:
# 从示例中提取特征列名称
FEATURE_COLUMNS = list(example.features.feature.keys())
if 'phrase' in FEATURE_COLUMNS:
    FEATURE_COLUMNS.remove('phrase')

# 重新计算各个索引
reset_hand_pose_index()

In [80]:
phrase_char_len_columns = [col for col in FEATURE_COLUMNS if 'right_hand' in col]
print(phrase_char_len_columns)

['x_right_hand_20', 'x_right_hand_18', 'x_right_hand_5', 'x_right_hand_4', 'y_right_hand_0', 'x_right_hand_15', 'y_right_hand_5', 'x_right_hand_0', 'z_right_hand_7', 'y_right_hand_6', 'y_right_hand_9', 'y_right_hand_18', 'y_right_hand_2', 'y_right_hand_4', 'y_right_hand_7', 'y_right_hand_15', 'x_right_hand_19', 'x_right_hand_13', 'y_right_hand_16', 'z_right_hand_4', 'y_right_hand_3', 'x_right_hand_8', 'x_right_hand_7', 'y_right_hand_8', 'y_right_hand_12', 'x_right_hand_10', 'x_right_hand_12', 'x_right_hand_16', 'y_right_hand_17', 'x_right_hand_11', 'x_right_hand_17', 'z_right_hand_17', 'x_right_hand_2', 'z_right_hand_5', 'y_right_hand_1', 'z_right_hand_8', 'z_right_hand_11', 'z_right_hand_16', 'z_right_hand_15', 'y_right_hand_19', 'z_right_hand_9', 'z_right_hand_3', 'y_right_hand_14', 'z_right_hand_12', 'x_right_hand_1', 'x_right_hand_9', 'x_right_hand_6', 'z_right_hand_14', 'z_right_hand_6', 'z_right_hand_0', 'z_right_hand_1', 'z_right_hand_13', 'y_right_hand_20', 'z_right_hand_20', '

In [81]:
tf_records = dataset_df.file_id.map(lambda x: f'{base_output_path}/preprocessed/{x}.tfrecord').unique()
print(f"List of {len(tf_records)} TFRecord files.")

List of 68 TFRecord files.


In [82]:
with open (f"{base_input_path}character_to_prediction_index.json", "r") as f:
    char_to_num = json.load(f)

# Add pad_token, start pointer and end pointer to the dict
pad_token = 'P'
start_token = '<'
end_token = '>'
pad_token_idx = 59
start_token_idx = 60
end_token_idx = 61

char_to_num[pad_token] = pad_token_idx
char_to_num[start_token] = start_token_idx
char_to_num[end_token] = end_token_idx
num_to_char = {j:i for i,j in char_to_num.items()}

In [112]:
# Function to resize and add padding.
# Reference: https://www.kaggle.com/code/irohith/aslfr-transformer/notebook
def resize_pad(X):
    if tf.shape(X)[0] < FRAME_LEN:
        X = tf.pad(X, ([[0, FRAME_LEN-tf.shape(X)[0]], [0, 0], [0, 0]]))
    else:
        X = tf.image.resize(X, (FRAME_LEN, tf.shape(X)[1]))
    return X

# Detect the dominant hand from the number of NaN values.
# Dominant hand will have less NaN values since it is in frame moving.
def pre_process(X):
    rhand = tf.gather(X, RHAND_IDX, axis=1)
    lhand = tf.gather(X, LHAND_IDX, axis=1)
    rpose = tf.gather(X, RPOSE_IDX, axis=1)
    lpose = tf.gather(X, LPOSE_IDX, axis=1)
    
    rnan_idx = tf.reduce_any(tf.math.is_nan(rhand), axis=1)
    lnan_idx = tf.reduce_any(tf.math.is_nan(lhand), axis=1)
    
    rnans = tf.math.count_nonzero(rnan_idx)
    lnans = tf.math.count_nonzero(lnan_idx)
    
    # For dominant hand
    if rnans > lnans:
        hand = lhand
        pose = lpose
        
        hand_x = hand[:, 0*(len(LHAND_IDX)//3) : 1*(len(LHAND_IDX)//3)]
        hand_y = hand[:, 1*(len(LHAND_IDX)//3) : 2*(len(LHAND_IDX)//3)]
        hand_z = hand[:, 2*(len(LHAND_IDX)//3) : 3*(len(LHAND_IDX)//3)]
        hand = tf.concat([1-hand_x, hand_y, hand_z], axis=1)
        
        pose_x = pose[:, 0*(len(LPOSE_IDX)//3) : 1*(len(LPOSE_IDX)//3)]
        pose_y = pose[:, 1*(len(LPOSE_IDX)//3) : 2*(len(LPOSE_IDX)//3)]
        pose_z = pose[:, 2*(len(LPOSE_IDX)//3) : 3*(len(LPOSE_IDX)//3)]
        pose = tf.concat([1-pose_x, pose_y, pose_z], axis=1)
    else:
        hand = rhand
        pose = rpose
    
    hand_x = hand[:, 0*(len(LHAND_IDX)//3) : 1*(len(LHAND_IDX)//3)]
    hand_y = hand[:, 1*(len(LHAND_IDX)//3) : 2*(len(LHAND_IDX)//3)]
    hand_z = hand[:, 2*(len(LHAND_IDX)//3) : 3*(len(LHAND_IDX)//3)]
    hand = tf.concat([hand_x[..., tf.newaxis], hand_y[..., tf.newaxis], hand_z[..., tf.newaxis]], axis=-1)
    
    mean = tf.math.reduce_mean(hand, axis=1)[:, tf.newaxis, :]
    std = tf.math.reduce_std(hand, axis=1)[:, tf.newaxis, :]
    hand = (hand - mean) / std

    pose_x = pose[:, 0*(len(LPOSE_IDX)//3) : 1*(len(LPOSE_IDX)//3)]
    pose_y = pose[:, 1*(len(LPOSE_IDX)//3) : 2*(len(LPOSE_IDX)//3)]
    pose_z = pose[:, 2*(len(LPOSE_IDX)//3) : 3*(len(LPOSE_IDX)//3)]
    pose = tf.concat([pose_x[..., tf.newaxis], pose_y[..., tf.newaxis], pose_z[..., tf.newaxis]], axis=-1)
    
    other_features = tf.gather(X, OTHER_FEATURES_IDX, axis=1)
    
    print("Hand shape:", tf.shape(hand))
    print("Pose shape:", tf.shape(pose))
    print("Other features shape:", tf.shape(other_features))
    other_features = tf.expand_dims(other_features, axis=-1)
    #other_features=tf.reshape(other_features, tf.shape(hand))
    
    # 获取hand和pose的轴1的尺寸
    feature_len = tf.shape(hand)[1]
    # 通过在轴1上重复other_features来调整other_features的形状
    other_features = tf.tile(other_features, [1, feature_len, 1])
    print("Other features shape - after adding 1 dim:", tf.shape(other_features))
    
    #* problem: bug here
    X = tf.concat([hand, pose, other_features], axis=1)
    X = resize_pad(X)
    
    X = tf.where(tf.math.is_nan(X), tf.zeros_like(X), X)
    X = tf.reshape(X, (FRAME_LEN, len(LHAND_IDX) + len(LPOSE_IDX)+ len(OTHER_FEATURES_IDX)))
    return X

In [92]:
def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COLUMNS}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    #landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COLUMNS])
    landmarks = [tf.sparse.to_dense(features[COL]) if isinstance(features[COL], tf.sparse.SparseTensor) else features[COL] for COL in FEATURE_COLUMNS]

    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)
    
    return landmarks, phrase

In [85]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(char_to_num.keys()),
        values=list(char_to_num.values()),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
)

def convert_fn(landmarks, phrase):
    # Add start and end pointers to phrase.
    phrase = start_token + phrase + end_token
    phrase = tf.strings.bytes_split(phrase)
    phrase = table.lookup(phrase)
    # Vectorize and add padding.
    phrase = tf.pad(phrase, paddings=[[0, 64 - tf.shape(phrase)[0]]], mode = 'CONSTANT',
                    constant_values = pad_token_idx)
    # Apply pre_process function to the landmarks.
    return pre_process(landmarks), phrase

In [113]:
batch_size = 64
train_len = int(0.8 * len(tf_records))

train_ds = tf.data.TFRecordDataset(tf_records[:train_len]).map(decode_fn).map(convert_fn).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE).cache()
valid_ds = tf.data.TFRecordDataset(tf_records[train_len:]).map(decode_fn).map(convert_fn).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE).cache()

Hand shape: Tensor("Shape_1:0", shape=(3,), dtype=int32)
Pose shape: Tensor("Shape_2:0", shape=(3,), dtype=int32)
Other features shape: Tensor("Shape_3:0", shape=(2,), dtype=int32)
Other features shape - after adding 1 dim: Tensor("Shape_5:0", shape=(3,), dtype=int32)


ValueError: in user code:

    File "/tmp/ipykernel_28/648965697.py", line 19, in convert_fn  *
        return pre_process(landmarks), phrase
    File "/tmp/ipykernel_28/1140560025.py", line 71, in pre_process  *
        X = tf.concat([hand, pose, other_features], axis=1)

    ValueError: Dimension 0 in both shapes must be equal, but are 3 and 1. Shapes are [3] and [1]. for '{{node concat_2}} = ConcatV2[N=3, T=DT_FLOAT, Tidx=DT_INT32](truediv, concat_1, Tile, concat_2/axis)' with input shapes: [?,21,3], [?,5,3], [?,84,1], [] and with computed input tensors: input[3] = <1>.


In [ ]:
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


class LandmarkEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
# Customized to add `training` variable
# Reference: https://www.kaggle.com/code/shlomoron/aslfr-a-simple-transformer/notebook

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [batch_size[..., tf.newaxis], tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target, training):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att, training = training))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out, training = training) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out, training = training))
        return ffn_out_norm


In [ ]:
# Customized to add edit_dist metric and training variable.
# Reference:
# https://www.kaggle.com/code/irohith/aslfr-transformer/notebook
# https://www.kaggle.com/code/shlomoron/aslfr-a-simple-transformer/notebook

class Transformer(keras.Model):
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=60,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.acc_metric = keras.metrics.Mean(name="edit_dist")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes

        self.enc_input = LandmarkEmbedding(num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target, training):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y, training)
        return y

    def call(self, inputs, training):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source, training)
        y = self.decode(x, target, training)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch[0]
        target = batch[1]

        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, pad_token_idx))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Computes the Levenshtein distance between sequences since the evaluation
        # metric for this contest is the normalized total levenshtein distance.
        edit_dist = tf.edit_distance(tf.sparse.from_dense(target), 
                                     tf.sparse.from_dense(tf.cast(tf.argmax(preds, axis=1), tf.int32)))
        edit_dist = tf.reduce_mean(edit_dist)
        self.acc_metric.update_state(edit_dist)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result(), "edit_dist": self.acc_metric.result()}

    def test_step(self, batch):        
        source = batch[0]
        target = batch[1]

        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, pad_token_idx))
        loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        # Computes the Levenshtein distance between sequences since the evaluation
        # metric for this contest is the normalized total levenshtein distance.
        edit_dist = tf.edit_distance(tf.sparse.from_dense(target), 
                                     tf.sparse.from_dense(tf.cast(tf.argmax(preds, axis=1), tf.int32)))
        edit_dist = tf.reduce_mean(edit_dist)
        self.acc_metric.update_state(edit_dist)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result(), "edit_dist": self.acc_metric.result()}

    def generate(self, source, target_start_token_idx):
        """Performs inference over one batch of inputs using greedy decoding."""
        bs = tf.shape(source)[0]
        enc = self.encoder(source, training = False)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input, training = False)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = logits[:, -1][..., tf.newaxis]
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input

In [ ]:
class DisplayOutputs(keras.callbacks.Callback):
    def __init__(
        self, batch, idx_to_token, target_start_token_idx=60, target_end_token_idx=61
    ):
        """Displays a batch of outputs after every 4 epoch

        Args:
            batch: A test batch
            idx_to_token: A List containing the vocabulary tokens corresponding to their indices
            target_start_token_idx: A start token index in the target vocabulary
            target_end_token_idx: An end token index in the target vocabulary
        """
        self.batch = batch
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 4 != 0:
            return
        source = self.batch[0]
        target = self.batch[1].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        for i in range(bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            print(f"target:     {target_text.replace('-','')}")
            print(f"prediction: {prediction}\n")

In [ ]:
%%time

# Transformer variables are customized from original keras tutorial to suit this dataset.
# Reference: https://www.kaggle.com/code/shlomoron/aslfr-a-simple-transformer/notebook
batch = next(iter(valid_ds))

# The vocabulary to convert predicted indices into characters
idx_to_char = list(char_to_num.keys())
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=char_to_num['<'], target_end_token_idx=char_to_num['>']
)  # set the arguments as per vocabulary index for '<' and '>'

model = Transformer(
    num_hid=200,
    num_head=4,
    num_feed_forward=400,
    source_maxlen = FRAME_LEN,
    target_maxlen=64,
    num_layers_enc=2,
    num_layers_dec=1,
    num_classes=62
)
loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, label_smoothing=0.1,
)


optimizer = keras.optimizers.Adam(0.0001)
model.compile(optimizer=optimizer, loss=loss_fn)

history = model.fit(train_ds, validation_data=valid_ds, callbacks=[display_cb], epochs=13)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training loss', 'val_loss'])

In [ ]:
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.target_start_token_idx = start_token_idx
        self.target_end_token_idx = end_token_idx
        # Load the feature generation and main models
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, len(FEATURE_COLUMNS)], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = tf.cast(inputs, tf.float32)
        x = x[None]
        x = tf.cond(tf.shape(x)[1] == 0, lambda: tf.zeros((1, 1, len(FEATURE_COLUMNS))), lambda: tf.identity(x))
        x = x[0]
        x = pre_process(x)
        x = x[None]
        x = self.model.generate(x, self.target_start_token_idx)
        x = x[0]
        idx = tf.argmax(tf.cast(tf.equal(x, self.target_end_token_idx), tf.int32))
        idx = tf.where(tf.math.less(idx, 1), tf.constant(2, dtype=tf.int64), idx)
        x = x[1:idx]
        x = tf.one_hot(x, 59)
        return {'outputs': x}
    
tflitemodel_base = TFLiteModel(model)

In [ ]:
model.save_weights("model.h5")

In [ ]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]#, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = keras_model_converter.convert()
with open(f'{base_output_path}model.tflite', 'wb') as f:
    f.write(tflite_model)
    
infargs = {"selected_columns" : FEATURE_COLUMNS}

with open('inference_args.json', "w") as json_file:
    json.dump(infargs, json_file)


In [ ]:
!zip submission.zip  './model.tflite' './inference_args.json'

In [ ]:
interpreter = tf.lite.Interpreter("model.tflite")

REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

with open (f"{base_input_path}character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {j:i for i,j in character_map.items()}

found_signatures = list(interpreter.get_signature_list().keys())

if REQUIRED_SIGNATURE not in found_signatures:
    raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=batch[0][0])
prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)])
print(prediction_str,'\n',batch[0][0])
